We'll try to fetch the artists and tracks I listened to in a specific period from lastfm and see if we can encirh this info with spotify data (because it only allows to 50 last tracks in a beta api)
After that we can try som simple ML models on it :D 

In [11]:
import requests as re
import numpy as np
import time
import pandas as pd
import datetime
from datetime import date, timedelta
from IPython.core.display import clear_output
import json
import requests_cache

### Pre-work

In [20]:
#Get the csv from the last request
played_tracks = pd.read_csv('played_tracks.csv')

In [21]:
played_tracks.sample(3)

,0,1,2
388,Kraftwerk,Prologue - 2009 Remaster,1603737695
10018,Descendents,I Wanna Be a Bear,1584465871
1932,Idles,A Hymn,1601042935


In [24]:
#search by track and artist name
    #first we'll search by track and then narrow down by artist name

#cleanup dataset fo the unique values
played_tracks.drop(played_tracks.columns[2], axis = 1, inplace = True)

## Spotify API

In [2]:
#create a request cache
requests_cache.install_cache('tags_for_tracks_cache')

In [ ]:
#Authentication - getting an access token

auth_response = re.post('https://accounts.spotify.com/api/token', {
    'grant_type': 'client_credentials',
    'client_id': '6cb782c1b0404843b3e5a06e8361cb6e',
    'client_secret': 'a32c28deab714fa89091395089cbca90',
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

#create a header with the access token
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}